In [1]:
from keras.models import Sequential, load_model
import pandas as pd
import numpy as np
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import random
import datetime

from Indicator import Indicator

def check_trend(df):
    df.loc[(df['Close'] > df['MA']) &
    ((df['Close'] > df['Close'].shift(1)) | (df['MA'] > df['MA'].shift(1))) &
    ((df['Close'] > df['Close'].shift(2)) | (df['MA'] > df['MA'].shift(2))) &
    ((df['Close'] > df['Close'].shift(3)) | (df['MA'] > df['MA'].shift(3))) &
    ((df['Close'] > df['Close'].shift(4)) | (df['MA'] > df['MA'].shift(4))) &
    ((df['Close'] > df['Close'].shift(5)) | (df['MA'] > df['MA'].shift(5))) , 'Trend'] = 'Up'

    df.loc[(df['Close'] < df['MA']) &
    ((df['Close'] < df['Close'].shift(1)) | (df['MA'] < df['MA'].shift(1))) &
    ((df['Close'] < df['Close'].shift(2)) | (df['MA'] < df['MA'].shift(2))) &
    ((df['Close'] < df['Close'].shift(3)) | (df['MA'] < df['MA'].shift(3))) &
    ((df['Close'] < df['Close'].shift(4)) | (df['MA'] < df['MA'].shift(4))) &
    ((df['Close'] < df['Close'].shift(5)) | (df['MA'] < df['MA'].shift(5))) , 'Trend'] = 'Down'
    
    df.loc[df['Trend'].isnull(), 'Ntrend'] = df['Trend'].shift(1)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull(),'Ntrend'] = df['Trend'].shift(2)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull() & df['Trend'].shift(2).isnull(),'Ntrend'] = df['Trend'].shift(3)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull() & df['Trend'].shift(2).isnull() & 
    df['Trend'].shift(3).isnull(),'Ntrend'] = df['Trend'].shift(4)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull() & df['Trend'].shift(2).isnull() & 
    df['Trend'].shift(3).isnull() & df['Trend'].shift(4).isnull(),'Ntrend'] = df['Trend'].shift(5)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull() & df['Trend'].shift(2).isnull() & 
    df['Trend'].shift(3).isnull() & df['Trend'].shift(4).isnull() & df['Trend'].shift(5).isnull(),'Ntrend'] = df['Trend'].shift(6)
    
    df['S1'] = df['Close'].shift(-1)
    df['S2'] = df['Close'].shift(-2)
    df.loc[df['S1'].isnull(),'S1'] = 0
    df.loc[df['S2'].isnull(),'S2'] = 0

   
    #Trading Signal (0,1)
    df.loc[(df['Trend'] == 'Up') | (df['Ntrend'] == 'Up'), 'Trading Signal'] = ((abs(df['Close'] - df[['Close','S1','S2']].min(axis = 1)) / 
    abs(df[['Close','S1','S2']].max(axis = 1) - df[['Close','S1','S2']].min(axis = 1)))*0.5) + 0.5
    df.loc[(df['Trend'] == 'Down') | (df['Ntrend'] == 'Down'), 'Trading Signal'] = ((abs(df['Close'] - df[['Close','S1','S2']].min(axis = 1)) / 
    abs(df[['Close','S1','S2']].max(axis = 1) - df[['Close','S1','S2']].min(axis = 1)))*0.5)
   
    df.loc[df['Trend'].isnull(),'Trend'] =df['Ntrend']
    
    df = df.drop(['Ntrend','S1','S2'],axis=1)
        
    return df

# def fillindicator(df):
#     indicator = Indicator()
#     train_set = indicator.MA(df,15)
#     train_set = indicator.MACD(df)
#     train_set = indicator.STOCHASTIC(df)
#     train_set = indicator.RSI(df)
#     train_set = indicator.WILLIAMSR(df)
#     train_set = indicator.DMI(df)
#     train_set = indicator.AROON(df)
#     train_set = indicator.ATR(df)
#     train_set = indicator.CCI(df)
#     train_set = indicator.OBV(df)
#     return train_set

def norm(df,col_name):# this will get columnname + _n
    #Normalize data each colunm in to range -1 to 1
    df[col_name+'_n'] = (df[col_name]-df[col_name].min())/(df[col_name].max()-df[col_name].min())
    return df

def signal_to_num(df):
    df.loc[df['Trend']=='Down','Trend'] = 0
    df.loc[df['Trend']=='Up','Trend'] = 1
    return df

def fillindicator(df):
    indicator = Indicator()
    train_set = indicator.MACD(df,12,26,9,1)#MACD 1
    train_set = indicator.MACD(df,40,60,34,2)#MACD 2
    train_set = indicator.DMI(df,14)
    train_set = indicator.STOCHRSI(df)
    train_set = indicator.AO(df)
    train_set = indicator.WILLIANSR(df)
    train_set = indicator.RSI(df)
    train_set = indicator.MA(df,26)
    train_set['MA-26'] = train_set['MA']
    train_set = indicator.MA(df,15)
    train_set = indicator.check_trend(df)
    return train_set

def shiftdata(df,col_name,sft):
    for i in range(sft):
        i+=1
        shiftname = col_name+'-'+str(i)
        df[shiftname] = df[col_name].shift(i)

    return df

def add_shift_day(data,day):
    r_list = []
    for i in range(day,len(data)):#Number mean day to shift
        r_list.append(data[i-day : i])
    return np.array(r_list)

In [20]:
start = datetime.datetime(2019,1,1)#26 day before start year for make indicator
end = datetime.datetime(2020,1,1)

money = 100000
stocklist = ['AES','EBAY','VRTX','GOOGL','IP','CSCO','UAL','NVR','TPR','BA']
# stock = 'KTC.bk'

In [28]:
#Benchmark
BM = []
for stock in stocklist:
    df = web.DataReader(stock,'yahoo',start,end)
    pf = (df['Close'][-1] - df['Close'][0])*money
    percent = ((df['Close'][-1] - df['Close'][0])/(df['Close'][0]))*100
    BM.append([stock,percent,1,1,pf])

In [22]:
# #Hybrid Stock Model
# HBstock = []
# for stock in stocklist:
#     df = web.DataReader(stock,'yahoo',start,end)
#     df = fillindicator(df)
#     df = norm(df,'MA')
#     df = norm(df,'MACD-1')
#     df = norm(df,'SIGNAL LINE1')
#     df = norm(df,'%K')
#     df = norm(df,'%D')
#     df = norm(df,'%R')
#     df = norm(df,'RSI-7')
#     df = norm(df,'RSI-14')
#     df = check_trend(df)
#     df = df.iloc[61:]
#     df = signal_to_num(df)

#     model = load_model('Model/best_4.h5')

#     test_x = df[['MA_n','MACD-1_n','SIGNAL LINE1_n','RSI-7_n','%K_n','%D_n','%R_n','Trading Signal','Trend']]
#     x_test = test_x[['MA_n','MACD-1_n','SIGNAL LINE1_n','RSI-7_n','%K_n','%D_n','%R_n','Trading Signal']].values
#     y_test = test_x['Trend'].values
#     x_test=np.asarray(x_test).astype(np.float32)
#     y_test=np.asarray(y_test).astype(np.float32)

#     pred = model.predict(x_test)

#     data = df
#     data['pred'] = pred
#     mean = data['pred'].mean()
#     data['pred'] =np.where(data['pred'] >= mean,'up','down')

#     data['up'] = np.where(data['pred']=='up',data['Close'],None)
#     data['down'] = np.where(data['pred']=='down',data['Close'],None)

#     b = []
#     s = []
#     flag = 'sell'
#     for i,r in data.iterrows():
#         if (flag=='sell') & (r['pred']=='up'):
#             b.append(r['Close'])
#             flag = 'buy'
#         elif (flag=='buy') & (r['pred']=='down'):
#             s.append(r['Close'])
#             flag = 'sell'

#     if len(b)>len(s):
#         del b[-1]

#     modelbuy = len(b)
#     modelsell = len(s)
#     pfmodel = sum((np.array(s)*money)-(np.array(b)*money))
#     profitmodel = []
#     for i in range(len(b)):
#         profitmodel.append(((s[i]*money-b[i]*money)/(b[i]*money))*100)
#     profitmodel = sum(profitmodel)
#     HBstock.append([stock,profitmodel,modelbuy,modelsell,pfmodel])

In [23]:
#MACD Simulation
MACD=[]
for stock in stocklist:
    df = web.DataReader(stock,'yahoo',start,end)
    df = fillindicator(df)
    df = df.iloc[61:]

    df['signal'] = 0
    df.loc[((df['MACD-1']>df['SIGNAL LINE1'])&(df['MACD-1'].shift()<df['SIGNAL LINE1'].shift())),'signal'] = 1
    df.loc[((df['MACD-1']<df['SIGNAL LINE1'])&(df['MACD-1'].shift()>df['SIGNAL LINE1'].shift())),'signal'] = 2

    prd = []
    for i ,row in df.iterrows():
        if row['signal']==1:
    #         plt.scatter(x=i, y=row['Close'], c='green')
            prd.append(('buy',row['Close']))
        elif row['signal']==2:
    #         plt.scatter(x=i, y=row['Close'], c='red')
            prd.append(('sell',row['Close']))

    if prd[0][0]=='sell':
        del prd[0]
    if len(prd) %2!=0:
        del prd[-1]

    b = []
    s = []
    for i in prd:
        if i[0] =='buy':
            b.append(i[1])
        else:
            s.append(i[1])
    macdbuy = len(b)
    macdsell = len(s)
    pfmacd= []
    profitmacd = []
    for i in range(len(b)):
        pfmacd.append(((s[i]-b[i])/(b[i]))*money)
        profitmacd.append(((s[i]-b[i])/(b[i]))*100)
    pfmacd = sum(pfmacd)
    MACD.append([stock,sum(profitmacd),macdbuy,macdsell,pfmacd])

In [24]:
#Project model
PJmodel = []
model = load_model('Model/model_lstm3_1_20.h5')
for stock in stocklist:
    df = web.DataReader(stock,'yahoo',start,end)
    df = fillindicator(df)
#     df['AO-signal'] = np.where(df['AO'] > df['AO'].shift(1),1,0)
    df.loc[df['Trend'] == 'Down', 'Trend'] = 0.1
    df.loc[df['Trend'] == 'Up', 'Trend'] = 0.9 

    #Delta
    df['MACD-SL-1'] = df['MACD-1'] - df['SIGNAL LINE1']
    df['MACD-SL-2'] = df['MACD-2'] - df['SIGNAL LINE2']
    df['DMI'] = df['plusDI'] - df['minusDI']
    df['STOCHRSI'] = df['%K'] - df['%D']
    df['MA-trend'] = df['MA-26'] - df['Close']

    #MACD
    df = norm(df,'MACD-SL-1')
    df = norm(df,'MACD-SL-2')
    df = norm(df,'MACD-1')
    df = norm(df,'MACD-2')
    df = norm(df,'SIGNAL LINE1')
    df = norm(df,'SIGNAL LINE2')

    #DMI
    df = norm(df,'plusDI')
    df = norm(df,'minusDI')
    df = norm(df,'DMI')

    #StochasticRSI
    df = norm(df,'%K')
    df = norm(df,'%D')
    df = norm(df,'STOCHRSI')

    #Other
    df = norm(df,'Close')
    df = norm(df,'%R')
    df = norm(df,'RSI')
    df = norm(df,'AO')
    df = norm(df,'MA-trend')

    df = df[63:]
    p_test = df
    x_test = p_test[['MACD-SL-1_n','DMI_n','MACD-SL-2_n','Close_n','AO_n','RSI_n','MA-trend']].values
    x_test = add_shift_day(x_test,30)
    y_pred = model.predict(x_test)
    y_pred = np.argmax(y_pred,axis = -1)
    p_test = p_test.iloc[30:]

    p_test['pred'] = y_pred
    p_test['pred_b'] = np.where(p_test['pred'] == 0,p_test['Close'],np.nan)
    p_test['pred_s'] = np.where(p_test['pred'] == 1,p_test['Close'],np.nan)
    b = []
    s = []
    flag = 'sell'
    for i,r in p_test.iterrows():
        if (flag=='sell') & (r['pred']==0):
#             plt.scatter(x=i, y=r['Close'], c='green')
            b.append(r['Close'])
            flag = 'buy'
        elif (flag=='buy') & (r['pred']==1):
#             plt.scatter(x=i, y=r['Close'], c='red')
            s.append(r['Close'])
            flag = 'sell'
    if len(b)>len(s):
        del b[-1]
    modelbuy = len(b)
    modelsell = len(s)
    pfmodel = []
    profitmodel = []
    for i in range(len(b)):
        pfmodel.append((s[i]-b[i])/(b[i])*money)
        profitmodel.append(((s[i]-b[i])/(b[i]))*100)
    PJmodel.append([stock,sum(profitmodel),modelbuy,modelsell,sum(pfmodel)])    

C:\Users\Lenovo\anaconda3\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Lenovo\anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Lenovo\anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [14]:
# print('MACD')
# print('Profit % = ',profitmacd,'\n','buycount = ',macdbuy,'sellcount =',macdsell,'\n','Profit',pfmacd)
# print('---------------------------------------------------------------------------------------------------')
# print('Hybrid Stock Algorithm')
# print('Profit % = ',profitmodel,'\n','buycount = ',modelbuy,'sellcount =',modelsell,'\n','Profit',pfmodel)

In [29]:
print('start',start,'end',end)
print('                  Stock,         Profit %,     nbuy,nsell,Profit')
for i in range(len(MACD)):
    print('BenchMark       ',BM[i])
#     print('Hybrid algorithm',HBstock[i])
    print('MACD indicator  ',MACD[i])
    print('Project Model   ',PJmodel[i])
    print('--------------------------------------------------------------------------------')

start 2019-01-01 00:00:00 end 2020-01-01 00:00:00
                  Stock,         Profit %,     nbuy,nsell,Profit
BenchMark        ['AES', 37.62102051197801, 1, 1, 543999.9580383301]
MACD indicator   ['AES', 20.222537463014895, 5, 5, 20222.537463014894]
Project Model    ['AES', 3.651115370154275, 1, 1, 3651.1153701542753]
--------------------------------------------------------------------------------
BenchMark        ['EBAY', 28.64268258973236, 1, 1, 804000.0915527344]
MACD indicator   ['EBAY', -10.49806885499856, 7, 7, -10498.06885499856]
Project Model    ['EBAY', 3.3482743146468485, 1, 1, 3348.2743146468483]
--------------------------------------------------------------------------------
BenchMark        ['VRTX', 32.12840991932667, 1, 1, 5323999.0234375]
MACD indicator   ['VRTX', 28.92108984590815, 6, 6, 28921.08984590815]
Project Model    ['VRTX', 31.418177329030854, 3, 3, 31418.177329030856]
--------------------------------------------------------------------------------
BenchMar